In [1]:
from pprint import pprint
import datetime
from audio_utils import AudioRankerSequential, wave_read_to_wav_bytes
from judge import BASE_SYSTEM_INSTRUCTION, JudgeBasics, JudgeRanking, run_speech_llm

In [10]:
import weave

# SPEECH_AUDIO_DATASET_URI = "weave:///wandb-voice-ai/voice-judge/object/generated_speech_audio:v1"
SPEECH_AUDIO_DATASET_URI = "weave:///wandb-voice-ai/voice-judge/object/generated_speech_audio_test"
N_SAMPLES_TO_RANK = 3

weave_client = weave.init("wandb-voice-ai/voice-judge")
print("Downloading speech samples from Weave...")
ds_ref = weave.ref(SPEECH_AUDIO_DATASET_URI).get()
speech_samples = list(ds_ref.rows)

samples_to_rank = {}
for i in range(N_SAMPLES_TO_RANK):
    samples_to_rank[speech_samples[i]["voice_instructions_id"]] = {
        "audio": speech_samples[i]["audio"],
        "audio_bytes": wave_read_to_wav_bytes(speech_samples[i]["audio"]),
        "instructions": speech_samples[i]["voice_instructions"],
        "short_hash": None,
        "pairwise_comparison_history": {},
    }

ids_to_rank = list(samples_to_rank.keys())

weave: Logged in as Weights & Biases user: morgan.
weave: View Weave data at https://wandb.ai/wandb-voice-ai/voice-judge/weave


In [11]:
sample_1_id = ids_to_rank[0]
sample_2_id = ids_to_rank[1]


@weave.op
def rank_audio(sample_one, sample_two):  # , sample_three):
    pass


_, call = rank_audio.call(
    samples_to_rank[sample_1_id], samples_to_rank[sample_2_id]
)  # "", samples_to_rank[2])
target_call_id = call.id

ranker = AudioRankerSequential(
    [
        {
            "id": sample_1_id,
            "audio": samples_to_rank[sample_1_id]["audio_bytes"],
            "original_input_order": 1,
        },
        {
            "id": sample_2_id,
            "audio": samples_to_rank[sample_2_id]["audio_bytes"],
            "original_input_order": 2,
        },
    ],
    weave_client=weave_client,
    target_call_id=target_call_id,
    image_path="assets/gta-vi_guy_in_front_of_truck_cropped.jpg",
)

weave: 🍩 https://wandb.ai/wandb-voice-ai/voice-judge/r/call/0197112b-283e-7493-ac83-b729c329a663


weave: 📦 Published to https://wandb.ai/wandb-voice-ai/voice-judge/weave/objects/AudioSequentialRanking/versions/0hd2Wj2WfhQyILdb85TPQUd5YBnY7agbbYYdFEo6X0E


In [12]:
ranker.display_widget()

In [13]:
ranking = ranker.get_final_rankings()
ranking

{'rankings': [{'rank': 1,
   'id': 'weary_georgia_mutter_20250526_2033',
   'original_input_order': 2,
   'short_hash': 'c8ab9b2d'},
  {'rank': 2,
   'id': 'surprised_french_veteran_20250526_2033',
   'original_input_order': 1,
   'short_hash': '39de6b56'}],
 'completed_at': '2025-05-27_10-55'}

In [14]:
for r in ranking["rankings"]:
    competitor_id = [rr.get("id") for rr in ranking["rankings"] if rr.get("id") != r.get("id")][0]
    samples_to_rank[r.get("id")]["pairwise_comparison_history"][ranking["completed_at"]] = {
        "competitor_id": competitor_id,
        "sample_rank_in_this_pair": r.get("rank"),
    }

print(samples_to_rank["weary_georgia_mutter_20250526_2033"]["pairwise_comparison_history"])
print(samples_to_rank["surprised_french_veteran_20250526_2033"]["pairwise_comparison_history"])

{'2025-05-27_10-55': {'competitor_id': 'surprised_french_veteran_20250526_2033', 'sample_rank_in_this_pair': 1}}
{'2025-05-27_10-55': {'competitor_id': 'weary_georgia_mutter_20250526_2033', 'sample_rank_in_this_pair': 2}}


In [13]:
samples_with_rankings = []
for r in final_rankings:
    sample_id = r.get("id")
    if sample_id in samples_to_rank_dict:
        s = samples_to_rank_dict[sample_id]
        samples_with_rankings.append(
            {
                "id": s.get("id"),
                "audio": s.get("audio"),
                "instructions": s.get("instructions"),
                "rank": r.get("rank"),
                "audio_bytes": wave_read_to_wav_bytes(s.get("audio")),
                "short_hash": r.get("short_hash"),
            }
        )

# sort the samples_with_rankings list by rank
samples_with_rankings.sort(key=lambda x: x.get("rank"))
samples_with_rankings

[{'id': 'surprised_french_veteran_20250526_2033',
  'audio': <wave.Wave_read at 0x12203f9d0>,
  'instructions': 'Voice/Affect: raspy, weather-worn male, late-50s, clear but noticeable French accent in English.\nTone & Emotion: loud, rapid-fire surprise laced with frustrated disbelief and dark humor.\nDelivery: fast, agitated, slightly breathless; brief pause after “up there?” then punch the rest.\nPronunciation/Emphasis: stress “GTA 6,” “by now,” and “hand.” Keep French vowels on “coding.”\nLine: “What are you doing up there? (short beat) GTA 6 was supposed to be out by now — you coding the whole thing by hand or what?!”',
  'rank': 1,
  'audio_bytes': b'RIFF\x84\x04\x05\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\xc0]\x00\x00\x80\xbb\x00\x00\x02\x00\x10\x00data`\x04\x05\x00\xfa\xff\x04\x00\xfd\xff\xfe\xff\xfa\xff\x01\x00\x00\x00\xfc\xff\xff\xff\xfb\xff\x00\x00\xfc\xff\x01\x00\x01\x00\x05\x00\x02\x00\x00\x00\x04\x00\xfe\xff\x07\x00\x02\x00\t\x00\x06\x00\t\x00\x07\x00\x06\x00\n\x00\x06\

In [13]:
samples_with_rankings = []
for r in final_rankings:
    for s in samples_to_rank:
        if s.get("id") == r.get("id"):
            samples_with_rankings.append(
                {
                    "id": s.get("id"),
                    "audio": s.get("audio"),
                    "instructions": s.get("instructions"),
                    "rank": r.get("rank"),
                    "audio_bytes": wave_read_to_wav_bytes(s.get("audio")),
                    "short_hash": r.get("short_hash"),
                }
            )

# sort the samples_with_rankings list by rank
samples_with_rankings.sort(key=lambda x: x.get("rank"))

In [ ]:
task_prompt = "Given a ranking of 3 voices by a human judge from most liked to least liked, \
describe the voice characteristics that lead to the ranking."


@weave.op
async def run_speech_analyzer():
    # Example with file path
    builder_result = await run_speech_llm(
        system_instruction=BASE_SYSTEM_INSTRUCTION,
        prompt=task_prompt,
        model_name="gemini-2.5-pro-preview-05-06",  # "gemini-2.0-flash",
        temperature=0.1,
        response_model=JudgeBasics,
        audio_data=[
            samples_with_rankings[0]["audio_bytes"],
            samples_with_rankings[1]["audio_bytes"],
            samples_with_rankings[2]["audio_bytes"],
        ],
    )
    return builder_result


# Run example
# result = asyncio.run(example())
builder_result = await run_speech_analyzer()
pprint(builder_result.thinking)
pprint(builder_result.how_to_judge_a_voice)

('The ranking seems to prioritize naturalness, clarity, and an engaging tone '
 "that fits the context of the dialogue (which sounds like a gamer's "
 'frustration). \n'
 'Rank 1 voice is clear, energetic, and has a natural, expressive intonation. '
 'The pitch is moderate and pleasant. It sounds like a young adult, fitting '
 'the context of talking about GTA 6. The delivery has a good balance of '
 'exasperation and clarity.\n'
 "Rank 2 voice is deeper and more measured. It's clear, but less energetic and "
 'expressive than Rank 1. The pacing is slightly slower. While a good voice, '
 'it might sound a bit too serious or gruff for the specific line compared to '
 'the more animated Rank 1. It has a slightly gravelly quality.\n'
 'Rank 3 voice is higher-pitched and sounds younger, almost adolescent. The '
 'delivery feels a bit more forced or theatrical, and less natural than the '
 "other two. The intonation is more pronounced but doesn't sound as authentic. "
 'It also sounds sligh

In [15]:
BASIC_JUDGE_SYSTEM_INSTRUCTION = "Assess the generated voices provided"
judge_prompt = f"""Based on the following criteria, your task is to rank the voices proivded:\

{str(builder_result.how_to_judge_a_voice)}

"""


@weave.op
async def run_speech_judge():
    # Example with file path
    result = await run_speech_llm(
        system_instruction=BASIC_JUDGE_SYSTEM_INSTRUCTION,
        prompt=judge_prompt,
        model_name="gemini-2.5-pro-preview-05-06",  # "gemini-2.0-flash",
        temperature=0.1,
        response_model=JudgeRanking,
        audio_data=[
            samples_with_rankings[0]["audio_bytes"],
            samples_with_rankings[1]["audio_bytes"],
            samples_with_rankings[2]["audio_bytes"],
        ],
        initial_audio_parts_prompt="\n\nvoice_1:\n",
        audio_parts_prompt_divider="\n\nvoice_{input_order}:\n",
    )
    return result


# Run example
# result = asyncio.run(example())
result = await run_speech_judge()
pprint(result.thinking)
pprint(result.ranking)
print()

# for s in samples_with_rankings:
#     print(s.get("id"), s.get("short_hash"))
# print()

print("Human Ranking:")
for i in range(3):
    pprint(f"Voice {i + 1}: {final_rankings[i]}")

('The primary goal is to rank voices based on their suitability as a '
 "'frustrated gamer' along with other vocal qualities. \n"
 '\n'
 'Voice 1: Exhibits excellent clarity and naturalness. The pitch is slightly '
 "higher and has a strained quality that perfectly matches the 'frustrated "
 "gamer' persona. Intonation effectively conveys rising frustration. Pacing is "
 'good, and energy is high and engaging. The timbre supports the emotional '
 'state. Highly suitable for the context.\n'
 '\n'
 'Voice 2: Clear and natural, but the pitch is lower and the delivery is '
 'calmer, sounding more like general annoyance or disappointment rather than '
 'acute gamer frustration. While expressive, the energy level is lower '
 'compared to the others for this specific context. Timbre is pleasant but '
 "doesn't convey the same level of agitation. Less suitable for a 'frustrated "
 "gamer' but good for a generally annoyed tone.\n"
 '\n'
 'Voice 3: Excellent clarity and naturalness. The pitch is